# **🧮 1. Simulaciones de Qubits con Retardo**

🎯 **Objetivo:**

* Comparar la dinámica de un qubit con y sin retardo usando un modelo de ecuaciones diferidas (DDE) y evaluar si hay diferencias significativas en los resultados mediante ANOVA.


📌 **Instrucciones:**

1. Instala y carga las librerías necesarias (`qutip`,` numpy`, `pandas`, `matplotlib`, etc.).
2. Completa y verifica el modelo de retardo (`exact_dde_step`) usando los parámetros dados.
3. Simula la dinámica de un qubit con retardo usando el esquema exacto.
4. Simula la dinámica sin retardo usando `QuTiP` y el operador de Pauli $𝜎𝑧$.
5. Prepara un DataFrame con los resultados para análisis estadístico.
6. Realiza un ANOVA para comparar ambos casos.
7. Grafica los resultados y analiza visualmente si hay diferencias relevantes.
8. Interpreta los resultados de la tabla ANOVA: ¿existe una diferencia estadísticamente significativa entre ambas simulaciones?


⚡ **Resultado Esperado:**

* Un gráfico con dos curvas: una para la simulación con retardo y otra sin retardo.
* Una tabla ANOVA que indique si hay diferencias significativas.
* Una reflexión escrita sobre la comparación entre ambos métodos.

## **🧮 Ejercicio de Simulaciones de Qubits con Retardo**

In [ ]:
# 1. Instalar librerías necesarias
# 💡 COMPLETAR
# --------------------------------------
# Escribe aquí el comando para instalar qutip:
!pip install _______________

# --------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 💡 COMPLETAR: importar qutip y statsmodels
import ____________ as qt
from ____________ import ols
import ____________ as sm
from scipy.integrate import quad
from scipy.special import comb
from math import factorial

# 2. Parámetros del modelo
alpha = -1.0
beta = 0.5
gamma = 0.75
tau = 1.0
h = tau / 10
N = int(tau / h)
t_max = 5 * tau
n_steps = int(t_max / h)

# 3. Funciones base del modelo
def phi(t):
    return (t + 1)**2

def phi_prime(t):
    return 2 * (t + 1)

def F(s):
    return gamma * phi_prime(s) + beta * phi(s)

# 4. Esquema DDE exacto
def exact_dde_step(x_prev, n, m):
    # 💡 COMPLETAR: primer término
    term1 = np.exp(________) * x_prev[n]

    term2 = 0
    for k in range(1, m):
        for r in range(1, k+1):
            binom = comb(k-1, r-1, exact=True)
            term2 += (gamma**(k-r)) * ((beta + alpha * gamma)**r) * (h**r / factorial(r)) * binom * x_prev[n - k*N]

    def integrand(s):
        inner_sum = gamma**(m-1)
        for k in range(1, m):
            for r in range(1, k+1):
                inner_sum += (gamma**(m-1-r)) * ((beta + alpha * gamma)**r) * (((n*h - m*tau + h - s)**r) / factorial(r))
        return inner_sum * np.exp(alpha * (n*h - m*tau + h - s)) * F(s)

    lower = n*h - m*tau
    upper = lower + h
    term3, _ = quad(integrand, lower, upper)

    return term1 + term2 + term3

# 5. Simulación con retardo
t = np.linspace(-tau, t_max, n_steps + N + 1)
x = np.zeros(len(t))

# 💡 COMPLETAR: aplicar condición inicial
for i in range(N+1):
    x[i] = ___________

# 💡 COMPLETAR: bucle de simulación DDE
for n in range(n_steps):
    m = int(np.floor((n*h)/tau) + 1)
    x[n + N + 1] = _______________________

# 6. Simulación sin retardo (QuTiP)
# 💡 COMPLETAR: definir Hamiltoniano y estado inicial
H = ___________
psi0 = ___________

# 💡 COMPLETAR: colapso y tiempo
c_ops = [np.sqrt(-alpha) * qt.sigmaz()]
t_qutip = t[t >= 0]

# 💡 COMPLETAR: simulación sin retardo
result_no_delay = qt.mesolve(H, psi0, t_qutip, c_ops=c_ops, e_ops=[qt.________()])
sz_no_delay = result_no_delay.expect[0]

# 7. Preparación para análisis ANOVA
x_delay = x[N + 1:N + 1 + len(sz_no_delay)]
min_len = min(len(x_delay), len(sz_no_delay), len(t_qutip))

# 💡 Cortar longitudes para comparación justa
x_delay = x_delay[:min_len]
sz_no_delay = sz_no_delay[:min_len]
t_qutip = t_qutip[:min_len]

# 8. DataFrame y ANOVA
# 💡 COMPLETAR
data = pd.DataFrame({
    'Tiempo': np.tile(t_qutip, 2),
    'Valor': np.concatenate([__________, __________]),
    'Grupo': ['Con Retardo'] * min_len + ['Sin Retardo'] * min_len
})

model = ols('Valor ~ C(Grupo)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# 9. Visualización
plt.figure(figsize=(10, 6))
plt.plot(t_qutip, x_delay, label='Con Retardo (DDE)', color='blue')
plt.plot(t_qutip, sz_no_delay, label='Sin Retardo (QuTiP)', color='red', linestyle='--')
plt.xlabel('Tiempo')
plt.ylabel('Expectativa ⟨σ_z⟩')
plt.title('Simulación de Qubit: Con vs. Sin Retardo')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 10. Mostrar tabla ANOVA
print("Tabla ANOVA:\n", anova_table)




---



# **🧩 2. Interpolación con Matriz de Vandermond**

🎯 **Objetivo:**

* Interpolar un polinomio de segundo grado que pase por tres puntos dados usando la matriz de Vandermonde.


📌 **Instrucciones:**

1. Define tres pares de puntos $(𝑥_𝑖,𝑦_𝑖)$ que serán usados para construir el polinomio.
2. Completa la línea de código que construye la matriz de Vandermonde:
`V = np.vander(x, increasing=True)`
3. Usa `np.linalg.solve()` para resolver el sistema lineal y obtener los coeficientes del polinomio.
4. Evalúa el polinomio sobre un rango de valores `x_plot` usando los coeficientes obtenidos.
5. Grafica el polinomio interpolado y los puntos originales.
6. Agrega una breve interpretación: ¿cómo cambia la curva si modificas los puntos?


⚡ **Resultado Esperado:**

* Un gráfico suave que pase exactamente por los tres puntos definidos.
* Visualización del polinomio y los puntos originales.
* Una reflexión sobre la forma del polinomio.

📊 **Datos experimentales**

| Tiempo (s) | Temperatura (°C) |
| ---------- | ---------------- |
| 0.0        | 22.0             |
| 1.0        | 24.5             |
| 2.0        | 29.0             |

## 🧩**Ejercicio con Vandermonde**

In [ ]:
# 💡 IMPORTAR LIBRERÍAS
import _____________ as np
import _____________ as plt

# 1. Datos experimentales
x = np.array([______, ______, ______])  # Variable Dependiente
y = np.array([______, ______, ______])  # Variable Independiente

# 2. Construcción de la matriz de Vandermonde
V = np.vander(x, ____________)  # True o False según el orden

# 3. Resolver sistema lineal
a = np.linalg.solve(_________, _________)

# 4. Mostrar coeficientes
print("Coeficientes del polinomio interpolador:")
for i, coef in enumerate(a):
    print(f"a{i} = {coef:.4f}")

# 5. Definir función polinomial
def P(t):
    return sum(_____________ for i in range(len(a)))

# 6. Evaluar y graficar
t_vals = np.linspace(min(x), max(x), 100)
y_vals = P(t_vals)

plt.plot(t_vals, y_vals, label='Polinomio Interpolador')
plt.plot(x, y, 'ro', label='Puntos Originales')
plt.xlabel(_____________) # Variable Dependiente (Magnitus, Unidad)
plt.ylabel(_____________) # Variable Independiente (Magnitus, Unidad)
plt.title('Interpolación con Matriz de Vandermonde')
plt.grid(True)
plt.legend()
plt.show()